식별된 PII 엔터티가 있으면 해당 엔터티에 대해 다양한 비식별화 작업을 수행할 수 있습니다. 지원되는 연산자에 대한 자세한 내용 은 익명화 문서 를 참조하십시오 .

익명화에는 각 엔터티 유형에 대해 요청된 작업을 지정하는 구성이 필요합니다. PII 엔터티를 엔터티 유형 이름으로 바꾸는 기본 연산자도 있습니다.

각 운영자는 작업(수정, 해시, 마스크, 바꾸기, 암호화 등)을 수행하는 데 필요한 매개변수가 있는 고유한 구성을 가지고 있습니다.

다음은 presidio-anonymizer를 사용하는 간단한 예입니다.

In [4]:
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import RecognizerResult

# Analyzer output
analyzer_results = [
    RecognizerResult(entity_type="PERSON", start=11, end=15, score=0.8),
    RecognizerResult(entity_type="PERSON", start=17, end=27, score=0.8),
]

# Initialize the engine:
engine = AnonymizerEngine()

# Invoke the anonymize function with the text,
# analyzer results (potentially coming from presidio-analyzer) and
# Operators to get the anonymization output:
result = engine.anonymize(
    text="My name is Bond, James Bond", analyzer_results=analyzer_results
)

print("De-identified text")
print(result.text)

De-identified text
My name is <PERSON>, <PERSON>


추가 연산자를 도입하기 위해 OperatorConfig. 이 예에서 우리는:

1. PHONE_NUMBER엔터티 의 마지막 12자를 마스킹하고 다음으로 바꿉니다.*

2. TITLE항목 수정
3. 다른 모든 엔터티를 문자열로 바꿉니다 <ANONYMIZED>.

전체 예:

In [7]:
from pprint import pprint
import json

from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig, RecognizerResult


# Analyzer output
analyzer_results = [
    RecognizerResult(entity_type="PERSON", start=11, end=15, score=0.8),
    RecognizerResult(entity_type="PERSON", start=17, end=27, score=0.8),
]

text_to_anonymize = "My name is Bond, James Bond"

anonymizer = AnonymizerEngine()

# Define anonymization operators
operators = {
    "DEFAULT": OperatorConfig("replace", {"new_value": "<ANONYMIZED>"}),
    "PHONE_NUMBER": OperatorConfig(
        "mask",
        {
            "type": "mask",
            "masking_char": "*",
            "chars_to_mask": 12,
            "from_end": True,
        },
    ),
    "TITLE": OperatorConfig("redact", {}),
}

anonymized_results = anonymizer.anonymize(
    text=text_to_anonymize, analyzer_results=analyzer_results, operators=operators
)

print(f"text: {anonymized_results.text}")
print("detailed result:")

pprint(json.loads(anonymized_results.to_json()))


text: My name is <ANONYMIZED>, <ANONYMIZED>
detailed result:
{'items': [{'end': 37,
            'entity_type': 'PERSON',
            'operator': 'replace',
            'start': 25,
            'text': '<ANONYMIZED>'},
           {'end': 23,
            'entity_type': 'PERSON',
            'operator': 'replace',
            'start': 11,
            'text': '<ANONYMIZED>'}],
 'text': 'My name is <ANONYMIZED>, <ANONYMIZED>'}
